<a href="https://colab.research.google.com/github/lizarurak/Rurak/blob/main/Prophet_pcs123.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

pip install prophet --quiet - встановіть через термінал.
pip install pandas
pip install matplotlib

In [ ]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt


Loading of DataSet, Some checks and data prep

In [ ]:
# Load dataset
df = pd.read_csv('/content/sample_data/dataset_pcs.csv')


In [ ]:
# Show first few rows
df.head()

,sales_date,sku_id,store_id,sales_count,promo,social_trend_score
0,12/6/2023,SKU_2516,16,14,False,"0,33"
1,4/5/2023,SKU_2516,1,10,True,"0,82"
2,6/20/2023,SKU_2516,8,5,False,"0,33"
3,1/5/2023,SKU_2516,9,14,False,"0,56"
4,4/23/2023,SKU_2516,9,12,False,"0,35"


In [ ]:
# Check the column names in your dataset
df.columns

Index(['sales_date', 'sku_id', 'store_id', 'sales_count', 'promo',
       'social_trend_score'],
      dtype='object')

In [ ]:
#  Convert 'period' to datetime
df['sales_date'] = pd.to_datetime(df['sales_date'])

In [ ]:
# Fix numeric columns with commas instead of dots
columns_to_fix = ['social_trend_score']

for col in columns_to_fix:
    df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [ ]:
# Select only required columns (adjust as needed)
df = df[['sales_date', 'sku_id', 'store_id', 'sales_count',
         'promo', 'social_trend_score']]


In [ ]:
# Prepare results container
forecast_all = []

In [ ]:
# Loop through each unique SKU
sku_list = df['sku_id'].unique()

for sku in sku_list:
    sku_df = df[df['sku_id'] == sku].copy()

    # Rename columns to match Prophet requirements
    sku_df.rename(columns={'sales_date': 'ds', 'sales_count': 'y'}, inplace=True)

    # Drop rows with missing values in regressors
    sku_df = sku_df.dropna(subset=['promo', 'social_trend_score'])

    # Initialize Prophet with additional regressors
    model = Prophet()
    model.add_regressor('promo')
    model.add_regressor('social_trend_score')

    model.fit(sku_df)

    # Create future dataframe for 28 days
    future = model.make_future_dataframe(periods=28)

    # Add future regressor values (using last known value)
    for col in ['promo','social_trend_score']:
        last_val = sku_df[col].iloc[-1]
        future[col] = last_val  # repeat last known value

    # Forecast
    forecast = model.predict(future)


    result = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(28).copy()
    result.rename(columns={
        'ds': 'date',
        'yhat': 'forecast_qty_total',
        'yhat_lower': 'forecast_qty_total_lower',
        'yhat_upper': 'forecast_qty_total_upper'
    }, inplace=True)


    result['sku_id'] = sku


    forecast_all.append(result)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpf2c7mpc9/8z8aof70.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpf2c7mpc9/1n01qtz5.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=46236', 'data', 'file=/tmp/tmpf2c7mpc9/8z8aof70.json', 'init=/tmp/tmpf2c7mpc9/1n01qtz5.json', 'output', 'file=/tmp/tmpf2c7mpc9/prophet_modelu1pqrq15/prophet_model-20250424075406.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
07:54:06 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:54:06 - cmdstanpy - INFO - Chain [1] done processing
INFO:c

Getting a result, saving to Csv.

In [ ]:
#  Combine all forecasts
final_forecast_df = pd.concat(forecast_all, ignore_index=True)


In [ ]:

# Save to CSV
final_forecast_df.to_csv('forecast_28_days_all_skus_with_regressors.csv', index=False)

In [ ]:

# Show sample of result
final_forecast_df.head()

,date,forecast_qty_total,forecast_qty_total_lower,forecast_qty_total_upper,sku_id
0,2023-12-07,36.194956,33.737729,38.513846,SKU_2516
1,2023-12-08,52.770555,50.381261,55.358344,SKU_2516
2,2023-12-09,25.931597,23.508888,28.426543,SKU_2516
3,2023-12-10,29.895088,27.343266,32.350871,SKU_2516
4,2023-12-11,64.664563,62.280339,67.059081,SKU_2516
